<center><h1>BoomBikes Demand Prediction</h1></center>

<h2>Problem Statement</h2>

A bike-sharing system is a service in which bikes are made available for shared use to individuals on a short term basis for a price or free. Many bike share systems allow people to borrow a bike from a "dock" which is usually computer-controlled wherein the user enters the payment information, and the system unlocks it. This bike can then be returned to another dock belonging to the same system.


A US bike-sharing provider **BoomBikes** has recently suffered considerable dips in their revenues due to the ongoing Corona pandemic. The company is finding it very difficult to sustain in the current market scenario. So, it has decided to come up with a mindful business plan to be able to accelerate its revenue as soon as the ongoing lockdown comes to an end, and the economy restores to a healthy state. 


In such an attempt, BoomBikes aspires to understand the demand for shared bikes among the people after this ongoing quarantine situation ends across the nation due to Covid-19. They have planned this to prepare themselves to cater to the people's needs once the situation gets better all around and stand out from other service providers and make huge profits.


They have contracted a consulting company to understand the factors on which the demand for these shared bikes depends. Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market. The company wants to know:

<ul>
    <li>Which variables are significant in predicting the demand for shared bikes.</li>
    <li>How well those variables describe the bike demands</li>
</ul>

Based on various meteorological surveys and people's styles, the service provider firm has gathered a large dataset on daily bike demands across the American market based on some factors. 


<h2>Business Goal:</h2>

You are required to model the demand for shared bikes with the available independent variables. It will be used by the management to understand how exactly the demands vary with different features. They can accordingly manipulate the business strategy to meet the demand levels and meet the customer's expectations. Further, the model will be a good way for management to understand the demand dynamics of a new market. 


<h2>Dataset characteristics:</h2>
<table>
    <tr><th style="text-align:center">instant</th><th style="text-align:center">record index</th></tr>
	<tr><td style="text-align:center">dteday</td><td style="text-align:left">date</td></tr>
	<tr><td style="text-align:center">season</td><td style="text-align:left">season (1:spring, 2:summer, 3:fall, 4:winter)</td></tr>
	<tr><td style="text-align:center">yr</td><td style="text-align:left">year (0: 2018, 1:2019)</td></tr>
	<tr><td style="text-align:center">mnth</td><td style="text-align:left">month ( 1 to 12)</td></tr>
	<tr><td style="text-align:center">holiday</td><td style="text-align:left">weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)</td></tr>
	<tr><td style="text-align:center">weekday</td><td style="text-align:left">day of the week</td></tr>
	<tr><td style="text-align:center">workingday</td><td style="text-align:left">if day is neither weekend nor holiday is 1, otherwise is 0.</td></tr>
	<tr><td style="text-align:center">weathersit</td><td style="text-align:left">
        1: Clear, Few clouds, Partly cloudy, Partly cloudy<br>
		2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist<br>
		3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds<br>
		4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog</td></tr>
	<tr><td style="text-align:center">temp</td><td style="text-align:left">temperature in Celsius</td></tr>
	<tr><td style="text-align:center">atemp</td><td style="text-align:left">feeling temperature in Celsius</td></tr>
	<tr><td style="text-align:center">hum</td><td style="text-align:left">humidity</td></tr>
	<tr><td style="text-align:center">windspeed</td><td style="text-align:left">wind speed</td></tr>
	<tr><td style="text-align:center">casual</td><td style="text-align:left">count of casual users</td></tr>
	<tr><td style="text-align:center">registered</td><td style="text-align:left">count of registered users</td></tr>
	<tr><td style="text-align:center">cnt</td><td style="text-align:left">count of total rental bikes including both casual and registered</td></tr>
    </table>

In [ ]:
# import required packages
import numpy as np
import pandas as pd
import random
import calendar
import warnings
import seaborn as sns
import os.path
import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors
from matplotlib import cm
from matplotlib.font_manager import FontProperties
from matplotlib.collections import LineCollection

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

import statsmodels.api as sm  
from statsmodels.stats.outliers_influence import variance_inflation_factor

# surpress warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# import data from csv file
df = pd.read_csv("/kaggle/input/boombikes/day.csv")

In [ ]:
# import font awesome file and check if file exist to prevent error on execution
font_exists = os.path.exists("/kaggle/input/boombikes-data/Font Awesome 5 Free-Solid-900.otf")
fp = FontProperties(fname=r"/kaggle/input/boombikes-data/Font Awesome 5 Free-Solid-900.otf") 

In [ ]:
#check the shape of the dataframe
df.shape

In [ ]:
# Preview the head of the datframe
df.head()

# Data Cleaning

In [ ]:
# check the datatypes and null columns for the given dataset
df.info()

We don't have any null values in any of the features.

Lets validate and convert the datatypes of columns to appropriate format.

In [ ]:
# converting dteday to datetime object
df.dteday = pd.to_datetime(df.dteday,format="%d-%m-%Y")

In [ ]:
# On Analysis the day of the weekday columns seems to be having Monday as the first day of the week. 
# In our case the weekday starts with Sunday = 0, Monday = 1 ... Saturday = 6. 
# So we have to map the number in ordered days from Sunday to Saturday.
# making sunday as firstday of the week
#
# df.weekday = (df.dteday.dt.weekday + 1 ) % 7
# df.weekday.head(7)
#
# commented and not performing the above operation as per analysis from the calendar for the years 2018 and 2019 
# workingday, weekdays and holiday features are incorrectly mapped for the dteday so it might mislead
# the data for all these columns, instead we will drop dteday.

In [ ]:
# dropping unncessary columns
df.drop(columns=['instant'],inplace=True) # instant is a record index variable which is not required for our analysis

In [ ]:
df.head()

Let look at the overview of the table value description

In [ ]:
df.describe()

All the data looks good and error free now we can proceed to data visualtisation step

We can now visualize the dataset to understand the data better and observe patterns to get more insights and product a meaningful output

## EDA

Lets perform some EDA on our dataset

In [ ]:
# let plot a pair plot to see how the relations are between feature values
sns.pairplot(df,vars=['season','temp','windspeed','casual','atemp','workingday','registered','weekday','mnth','yr','holiday','cnt']);

In [ ]:
# set figure size
plt.figure(figsize=(10,10))

# plot heatmap for the correlation
sns.heatmap(round(df.corr(),2),annot=True,cmap="viridis");

**Drop Temp feature**

We can see that there is a high correlation between temp and atemp so lets drop temp feature from the dataset to prevent multicollinearity

In [ ]:
columns_to_drop = []
columns_to_drop.append('temp')

## Convert to categorical columns

We have few categorical columns with numerical values. It would be better to apply the conventional name for visualisation

## Weekday

In [ ]:
# assigning day names to day number of the week
# 0 sunday
# 1 monday
# ...
# 6 saturday

df.weekday = df.weekday.apply(lambda x: calendar.day_abbr[x-1]).astype('category')
df.weekday

## Month

In [ ]:
# set month abbrivation for month number
month_name = {i : calendar.month_abbr[i] for i in np.arange(1,13)}

# map the month names in the data set mnth column and set to categorical datatype
df.mnth = pd.Categorical(df.mnth.map(month_name),categories=[calendar.month_abbr[i] for i in np.arange(1,13)],ordered=True)

# deleting month_name variable as it is not required anymore
del month_name

We have four season numbers which can be changed to categorical column but before that lets name the feature values

## Season

In [ ]:
# Create season mapping variable
season_name  = {1:'spring',2:'summer',3:'fall',4:'winter'}

# Mapping Season names for the number
df.season = df.season.map(season_name).astype("category")

# deleting season_name variable as it is not required anymore
del season_name

We have four type of weather that are recorder in numbers which can be added as a categorical feature. Hence lets name the feature values

## Weather

In [ ]:
# Create weather mapping variable
weather_name = {1:'clear',2:'cloudy',3:'rain',4:'snow'}

# Mapping Season names for the number
df.weathersit = df.weathersit.map(weather_name).astype("category")

# deleting weather_name variable as it is not required anymore
del weather_name

## Creating derived column week_month

In [ ]:
def week_in_month(dt):
    weekday_one = datetime.date(dt.year, dt.month, 1).weekday()
    weekday_day = datetime.date(dt.year, dt.month, dt.day).weekday()
    x=(dt.day - 1)//7 + 1 + (weekday_day < weekday_one)
    #print(x,' ',dt)
    return 'week '+str(x)

In [ ]:
df['week_month']=df['dteday'].apply(week_in_month).astype('category')
df['week_month']

In [ ]:
df.head()

In [ ]:
# view the columns names to pick the categorical columns
df.info()

# Data Visualisation

In [ ]:
categorical_features = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday','weathersit','week_month']

In [ ]:
#visualising categorical variables in our dataset with count
plt.figure(figsize=(30, 20))

# iterate over each column and plot a box plot
for i,v in enumerate(categorical_features):
    plt.subplot(3,3,i+1)
    sns.boxplot(x = v , y = 'cnt', data = df,palette='viridis')
plt.suptitle("Box Plot for all categorical features",size=25,y=0.95)
plt.show()

In [ ]:
# copy original dataset to temp dataset for visualization purpose
df_temp = df.copy()
# change date format
df_temp.dteday = pd.to_datetime(df_temp.dteday.dt.strftime('%d-%m'),format='%d-%m')
# create month locatore object
months_locator = mdates.MonthLocator()
# seperate the dataset based on year to temporary dataset
df_2018 = df_temp[df_temp['yr']==0]
df_2019 = df_temp[df_temp['yr']==1]

In [ ]:
# create figure size
plt.figure(figsize=(15,5))

# creata scatter plot for plotinh holidays in months
plt.scatter('dteday','cnt',data=df_temp[df_temp['holiday']==1],c='r',label='holiday')

# plot the graph for the yaer 2018 and 2019 based on cnt on each day
plt.plot('dteday','cnt',data=df_2018,label='2018')
plt.plot('dteday','cnt',data=df_2019,label='2019')

# set the majorlocator to months instead of day
plt.gca().xaxis.set_major_locator(months_locator)

# set tick labels and length of ticks
plt.gca().set_xticklabels([calendar.month_abbr[i] for i in np.arange(1,13)])
plt.gca().tick_params(axis='x',length=0)

# plot legend title and lebels
plt.legend()
plt.title("2018 vs 2019 bike sharing")
plt.ylabel("Count")
plt.xlabel("Days(Months)")

# show plot
plt.show()

**Observation**

We can see that moslty when its a holiday the bike sharing is lower.

The bike sharing has increased in the year 2019 compared to its previous year

The trend looks alike on the increaseing and decreasing points over the period of time in a year

In [ ]:
# define a method to plot graph for differnet years in a seperate plot
def plot_temp(temp_df,year):
    # create subplot with figuersize 15 width and 5 height
    plt.subplots(figsize=(15,5))
    
    # convert date object to number via matplotlib dates library
    x=mdates.date2num(temp_df.dteday)
    y=temp_df.cnt.values
    c=temp_df['atemp'].values
    
    # combine value of x and y to an array and Transform it to rows and reshape it to 3d array
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    
    # connect the point with next point in series to make a connect line between the scatter points.
    # ie paring the points to make each point to plot a line
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    # plot the graph with color map and norm to set color scale for temperature
    lc = LineCollection(segments,cmap=plt.get_cmap('coolwarm'), norm=plt.Normalize(3.5, 42.5))
    
    # set the array with temperature values
    lc.set_array(c)
    
    # set the line width property
    lc.set_linewidth(2)
    ax=plt.gca()
    fig=plt.gcf()
    
    # add collection event to axis object
    ax.add_collection(lc)
    
    # create color bar for the line collection
    axcb = fig.colorbar(lc)
    
    # set min and max limit for x and y axis
    plt.xlim(min(x), max(x))
    plt.ylim(min(y), max(y))
    
    # set major locator and its format
    ax.xaxis.set_major_locator(months_locator)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m'))
    
    # set ticke labels for the major locators
    ax.set_xticklabels([calendar.month_abbr[i] for i in np.arange(1,13)])
    ax.tick_params(axis='x',length=0)

    # set title and labes 
    plt.title("Bikes booked for the Year "+year+" (colored in Actual Temperature)")
    plt.xlabel("Months")
    plt.ylabel("Booked count")
    axcb.set_label('Actual Temperature')
    
    # set the layout to fit
    plt.tight_layout()
    
    # show plot
    plt.show()

In [ ]:
# plotting graphs
plot_temp(df_2018,'2018')
plot_temp(df_2019,'2019')

# **Observation**

We can see that when the bike sharing is comparitevly low  in lower temperature than in higher temperatures for the years 2018 and 2019

In [ ]:
# delete dataframe that is not need to save memeory
del df_temp;
del df_2018;
del df_2019;

In [ ]:
# count plot based on weather condition
_, ax = plt.subplots(figsize=(15,5))

# plot graph for weather vs count
(df.groupby('weathersit')[['casual','registered']].sum()/1000).plot(kind='bar',ax=ax)

# plot icons if font awesome exists
if font_exists:
    ax.text(0, 1500, "\uf185", fontproperties=fp, size=50, color="orange", ha="center", va="center")
    ax.text(1, 1250, "\uf6c4", fontproperties=fp, size=50, color="lightslategrey", ha="center", va="center")
    ax.text(2, 1000, "\uf740", fontproperties=fp, size=50, color="lightskyblue", ha="center", va="center")
else:
    print("please extract 'Font Awesome 5 Free-Solid-900.otf' from zip")

# remove borders    
sns.despine(top=True,right=True)

# set axis, labels and title
plt.ylabel("Count in Thousands")
plt.xlabel("Weather")
plt.title("Number of bikes booked\n in defferent weather")

# plot values for bars
for patch in ax.patches:
    ax.text(patch.get_x()+0.125,patch.get_height()+10,int(patch.get_height()),ha='center')

# show plot
plt.show();

**Observation**

The graph depicts that the bike sharing is less on rainy days and more on days in clear sky

In [ ]:
# craete a temporary df for month data count
month_data = (df.groupby('mnth')['cnt'].sum()/1000).to_frame().reset_index()

In [ ]:
# create subplots
fig , ax = plt.subplots(figsize=(12,6))

# check if font awesome file exists
if font_exists:
    # plot the graph for total counts in month
    plt.plot(month_data['cnt'],linewidth=5,c='saddlebrown',zorder=1);
    
    # plot bike direction to indicate increase or decrease in trend
    ax.text(1,175,"\uf84a",rotation=60,fontproperties=fp,size=20)
    ax.text(4,338,"\uf84a",rotation=18,fontproperties=fp,size=20)
    ax.text(9.355,290,"\uf84a",rotation=-50,fontproperties=fp,size=20)
    
    # fill color below and above the slope
    ax.fill_between(month_data['mnth'],month_data['cnt'],color='forestgreen')
    ax.fill_between(month_data['mnth'],np.ones(len(month_data['mnth']))*400,color='lightskyblue',alpha=0.5)
    
    # remove margin spacing
    plt.margins(0,0)
else:
    print("please extract 'Font Awesome 5 Free-Solid-900.otf' from zip")
    # plot a normal graph
    plt.plot(month_data['cnt']);
    
# set liimit for y axis
ax.set_ylim(100,400)

# remove axis border
sns.despine(top=True,right=True)

# set axis labels and title
plt.ylabel("Total Count in Thousands",size=15)
plt.xlabel("Months",size=15)
plt.title("Bike shares trend in each month",size=20);

# set tick values
ax.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11])
ax.set_xticklabels([calendar.month_abbr[i] for i in np.arange(1,13)]);
ax.set_yticks([100,150,200,250,300,350,400]);
ax.set_yticklabels([100,150,200,250,300,350,400]);
plt.tight_layout()

# show plot
plt.show()

<h3>Observation</h3>

The bike shares increases from Jan to May and starts decreasing from the month of September.

So we could say that the first half of the year undergoes a increase in bookings and after september the bike booking startes depleating

In [ ]:
# delete the temporary dataframe
del month_data

In [ ]:
sns.set_style('darkgrid')

In [ ]:
(df.groupby('weekday')['cnt'].sum()/1000).plot(kind='bar', color=list(sns.color_palette('summer',7)));
plt.title("Weeday vs Cnt")
plt.ylabel("Numer of boooking(Thousands)")
plt.xlabel("Weekday");

**Observation**

Weekday Doesn't show much difference so it can not be used for our model

In [ ]:
(df.groupby('week_month')['cnt'].sum()/1000).plot(kind='bar',color=list(sns.color_palette('summer',6)));
plt.title("Week of Month vs Cnt")
plt.ylabel("Number of bookings(Thousands)")
plt.xlabel("Week of Month");

In [ ]:
(df.groupby('workingday')['cnt'].sum()/1000).plot(kind='bar',color=['yellowgreen','darkgreen']);
plt.title("Workingday vs Cnt")
plt.ylabel("Number of bookings(Thousands)")
plt.xlabel("Workingday");

**Observation**

Bike booking is more on workingdays

In [ ]:
plt.style.use('default')

# creata subplot with figure size 15,6
fig , ax = plt.subplots(figsize=(15,6))

# create a plot for total count in season
(df.groupby('season')['cnt'].sum()/1000).plot(kind='barh',ax=ax,width=0.5,color=['burlywood','darkgreen','saddlebrown','lightsteelblue']);

# set x-axis limit
plt.xlim(0,1200)

# plot bar values 
for patch in ax.patches:
    ax.text(patch.get_width()+30,patch.get_y()+0.2,"{0:.0f} K".format(int(patch.get_width())),ha='center',size=12)
    
ax.tick_params(axis='both',labelsize=15,length=0)
plt.ylabel('Season',size=17)
plt.xlabel('Count',size=17)
plt.title('Number of bookings in Season',size=20)    

sns.despine(left=True,right=True,top=True,bottom=True)

if font_exists:
    ax.text(125,3.3,"\uf84a",fontproperties=fp,size=20)
    ax.text(200,2.29,"\uf84a",fontproperties=fp,size=20)
    ax.text(75,1.3,"\uf84a",fontproperties=fp,size=20)
    ax.text(300,0.3,"\uf84a",fontproperties=fp,size=20)
    
    ax.text(random.randrange(800),2.5,"\uf185",fontproperties=fp,size=20,color='orange')
    
    for i in np.arange(1,100):
        ax.text(random.randrange(800),3.06,"\uf7ad",fontproperties=fp,size=20,zorder=2,color='lightskyblue')
        ax.text(random.randrange(830),random.uniform(3.1, 3.5),"\uf2dc",fontproperties=fp,size=5,zorder=2,color='lightskyblue')
    for i in np.arange(1,20):
        ax.text(random.randrange(900),2.28,"\uf1bb",fontproperties=fp,size=20,zorder=random.randrange(2),color='g')
    for i in np.arange(1,10):
        ax.text(random.randrange(450),1.25,"\uf5bb",fontproperties=fp,size=13,zorder=random.randrange(2),color='tomato')
    for i in np.arange(1,50):        
        ax.text(random.randrange(50),1.25,"\uf773",fontproperties=fp,size=5,zorder=random.randrange(2),color='cornflowerblue')
        ax.text(random.randint(300, 350),1.235,"\uf773",fontproperties=fp,size=5,zorder=random.randrange(2),color='cornflowerblue')
    for i in np.arange(1,50):
        ax.text(random.randrange(1030),0.15,"\uf06c",rotation=random.randrange(360),fontproperties=fp,size=13,zorder=random.randrange(2),color='orange')
        ax.text(random.randrange(1030),0.15,"\uf06c",rotation=random.randrange(360),fontproperties=fp,size=13,zorder=random.randrange(2),color='darkorange')


**Drop casual and registered feature**

There is also a high correlation between registered and cnt. Since cnt is the sum of registered and casual we can drop these two features as well.

## Dropping the columns to Final Dataframe

We need to preserve the original dataset so we are dropping the columns and assigning it to a new dataframe

In [ ]:
columns_to_drop.extend(['dteday','casual','registered'])
columns_to_drop

In [ ]:
# Droppping the columns as it is not required for our Predicting model
df_final = df.drop(columns=columns_to_drop).copy()

In [ ]:
# change the oridnal categorical variable back to default category
df_final.mnth = df_final.mnth.astype('object').astype('category')

# Data Preparation

Before we start with our Regression model we have to fix our data for our regression model to fit the line for the feature values.

We have most of our data as categorical features which needs to be converted to numberical values. Factorizing the features wouldn't be good idea reather we could create dummy variables for the categorical columns

In [ ]:
df_final.info()

In [ ]:
df_final.head()

From the dataset we can see that season,mnth,weekday,weathersit has different levels that can be converted to dummy variables. 

In [ ]:
# getting the list of dummy variables
dummy_vars = ['season','mnth','weekday','weathersit','week_month']

# creating dummy vairable for the selected categories
for feature in dummy_vars:
    # dropping the first column from the dummy feature as it would give an redundant correlation
    dummy = pd.get_dummies(df_final[feature],drop_first=True)
    df_final = pd.concat([df_final,dummy],axis=1)
df_final.head()

We can now remove the original categorical features

In [ ]:
# droping categorical features
df_final.drop(dummy_vars,axis=1,inplace=True)

In [ ]:
df_final.info()

WE have converted all our categorical data into numerical values with one-hot encoding. This will help in regression

# Prepare Dataset for Model

As the data is ready our first step is to perform test-train split

In [ ]:
# to keep our trianing set split on same data set we can specify rand seed sonstant
np.random.seed(0)

# lets split our data set into 75% train and 25% test data
df_train, df_test = train_test_split(df_final,test_size=0.25,random_state=100) # specifying either on one size is enough

In [ ]:
# preview the train set
df_train.head()

In [ ]:
# preview the test set
df_test.head()

In [ ]:
df_train.shape,df_test.shape

## Scaling the features

we need to scale few features as they are not in a comaprable range. If we miss to scale the coefficeints obtained will be very large/small to other feature coeffiecients.

In [ ]:
# create scaler object
scaler = MinMaxScaler()

# picking the features that has more varience in range
numerical_features = ['atemp','hum','windspeed','cnt']

# fitting the dataset to get min-max
scaler.fit(df_train[numerical_features])

# performing scaler transform
df_train[numerical_features] = scaler.transform(df_train[numerical_features])

In [ ]:
# preview the scaled data in train set 
df_train[numerical_features].head()

In [ ]:
# statisticl view of the scaled data in train set 
df_train[numerical_features].describe()

### Creating predictor X and Target variable y

In [ ]:
# pop target varaible and set it to target training data set
y_train = df_train.pop('cnt')
# set the training data set for predictor variables
X_train = df_train

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Feature Selection

In [ ]:
# check the total number of features
len(df_train.columns)

we have 28 columns but we dont neeed all the columns to bild a good model it may lead to overfitting.

First lets build the linear regressor to find the intercept and coefficient and fit it to RFE

## Linear Regression Model

In [ ]:
# Create linear Regression Model
lm = LinearRegression()

# fit the data to our model
lm.fit(X_train,y_train);

In [ ]:
# view the intercept
lm.intercept_

In [ ]:
# view the coefficients
lm.coef_

Lets create an **RFE** model to automate the feature selection for our dataset that helps us identify the features that has higher impact on our target variable. Using RFE we can reduce the number of features to **15**

# RFE

In [ ]:
# create RFE for our Linear Regressor Model
rfe = RFE(lm,15)

# fit the train data to RFE
rfe = rfe.fit(X_train,y_train)

Lets view the feature rank the rank with number greater than one can be dropped which is shown in the selected column .ie the column selected is provided with a flag True is selected and false is rejected

In [ ]:
# print the rank of each features
pd.DataFrame(zip(X_train.columns,rfe.support_,rfe.ranking_),columns=['Feature','Selected','Rank']).sort_values('Rank')

In [ ]:
# lest extract the selected columns by RFE
selected_columns = X_train.columns[rfe.support_]
selected_columns

In [ ]:
# rejected columns suggested by RFE in our training data set
X_train.columns[~rfe.support_]

## Build Model

Lets sort the columns based on correlation value to add the best correlated column to the model one by one

In [ ]:
# creating a correlation list for the selected features with cnt(target variable) 
corr_cols = list(selected_columns)
corr_cols.append('cnt')
corr_cols = df_final[corr_cols].corr().iloc[:,-1:].sort_values('cnt',ascending=False).drop('cnt',axis=0)
corr_cols = corr_cols.reset_index()
corr_cols = corr_cols[corr_cols.cnt>0].append(corr_cols[corr_cols.cnt<0].sort_values('cnt'))

# preview the result
corr_cols

### Build Model in Statsmodel API for detailed analysis

Now lets build this model for the selected features using Stats package

Creating predefind function to make it reusable code

**OLS model**

In [ ]:
def build_model(X_train_rfe):
    # adding a constant variable for intercept
    X_train_rfe = sm.add_constant(X_train_rfe)

    # Initialize an OLS model for our dataset and fit the data to model
    lm = sm.OLS(y_train,X_train_rfe).fit()

    # view the summary of the model for selected features
    print(lm.summary())

    return lm

### VIF Analysis
The VIF score should be below 5 for a good model so lets remove the features one by one

In [ ]:
def VIF(X_train_rfe):
    # create a dummy dataframe
    vif = pd.DataFrame()
    
    # extract the column values to vif features column value
    vif['Features'] = X_train_rfe.columns
    
    # calculate vif for the train data for the added features
    vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
    
    # round the value to 2 decimals
    vif['VIF'] = round(vif['VIF'], 2)
    
    # sort values by hightevif value first
    vif = vif.sort_values(by = "VIF", ascending = False)
    
    # print vif table
    display(vif)
    
    # retrun vif object
    return vif

## Selecting an Approach to build the model

We have two way to build a model to find the best features selected by RFE that would fit.
<ul><li>Dropping a feature one by one from the model built with 15 features until it shows good performane without overfitting</li><li>Adding a feature one by one to the model until it shows a good performance metrics</li></ul>

Lets add features one by one and build our model

# Logic to build with best features

Lets create a custom logic based on the below condition

<table>
<tr><th style="text-align:center">Order</th><th style="text-align:center">P-value</th><th style="text-align:center">VIF</th><th style="text-align:center">Action</th></tr>
<tr><td>1</td><td>High</td><td>High</td><td>Drop these columns First</td></tr>
<tr><td>2</td><td>High</td><td>Low</td><td>Drop these columns one by one, because this could lower the VIF values of other columns to prevent it from being dropped in next step </td></tr>
<tr><td>3</td><td>Low</td><td>High</td><td>Drop the colums with VIF greater than 5</td></tr>
<tr><td>4</td><td>Low</td><td>Low</td><td>Keep these features</td></tr>
</table>	

In [ ]:
# create a empty data frame for xtrain and vif
X_train_rfe = pd.DataFrame()
vif = pd.DataFrame()

# creating this object to ignore vif for a single feature
count = 1

# created this varible to stop the outer loop of adding futher features for model
stop = False

# iteratre over the columns in order
for i,v in corr_cols.iterrows():
    
    # extract the first column name to be added
    col = v.values[0]
    
    # add the column to the traing data set
    X_train_rfe[col] = X_train[col]
    
    # rebuild the model again to ckeck for high vifs and p-values 
    # once a feature is dropped on the above conditions after adding 
    # the new feature from the previous step to the model
    while True:
        # build the model
        lm = build_model(X_train_rfe)
        
        # ignore vif and p-value check since there will
        # be only 1 column on first iteration
        if count != 1:
            
            # calculate VIF
            vif = VIF(X_train_rfe)

            # if the model reaches required r2 score stop the model from executing furher steps
            if lm.rsquared >= 0.80:
                stop = True
                break
            
            # Check if the p-value if high
            if (lm.pvalues > 0.05).sum() > 0:
                
                # extract feature fo high p-value
                feature = lm.pvalues[lm.pvalues > 0.05].index
                
                # check if this feature is not const
                if feature[0] != 'const':
                    
                    # if the VIF value is aslo high drop this columns first
                    if feature[0] in vif.loc[vif.VIF > 5,'Features']:
                        X_train_rfe.drop(feature[0],axis=1,inplace=True)                # order 1
                    else:
                        # if only the p-value is high drop it
                        X_train_rfe.drop(feature[0],axis=1,inplace=True)                # order 2
                
                # if the p-value column is 2nd in the list extract 
                # that feature name to drop if from dataset if there is 
                # a third value with high p-value it will be
                # validated in the next loop after rebuild on dropping the current feature
                elif (feature[0] == 'const') & (len(feature) > 1):
                    X_train_rfe.drop(feature[1],axis=1,inplace=True)                    # order 2
            
            # if VIF value is high drop it
            if (vif.VIF > 5).sum() > 0:
                X_train_rfe.drop(vif.loc[vif.VIF > 5,'Features'],axis=1,inplace=True)   # order 3
            else:
                break                                                                   # order 4
        else:
            break
    # stop the process
    if stop:
        break
    
    # increment count on adding new feature
    count = count + 1

The iteration stops when the model reaches above 0.8 but on the last loop of iteration we can see that atemp VIF increased above 5 once the feature windspeed was added, so instead of dropping atemp lets drop **windspeed** as atemp has higher impact/explaenation to the model.
And Lets add the next feature from the corr_cols list

## Manual Feature selection testing

In [ ]:
try:
    # dropping windspeed as it influenses other variable's VIF with less contribution on model output
    X_train_vif = X_train_rfe.drop('windspeed',axis=1)
except:
    print("windspeed is not available in the train dataset")    

Lets add the next feature after windspeed from the corr_cols which is **cloudy**

In [ ]:
# adding cloudy to the dataset for testing the model output
X_train_vif['cloudy'] = X_train['cloudy']

In [ ]:
# build the model
lm1 = build_model(X_train_vif);

In [ ]:
# check the VIF for our new model
_ = VIF(X_train_vif)

**Observation**

The model has a good r-square value but it feels like we have overfitted the model with too much columns lets try to remove two columns from the training dataset

## Dropping few columns to prevent overfitting of the model

lets try some manual dropping as **trial and error** method to see if dropping these columns does not affect the model and provided decent output.



we have three freatures from same category in our final model **summer, winter and spring**. It would be fine to drop one of them

In [ ]:
try:
    # dropping summer to the dataset for testing the model output
    X_train_vif1 = X_train_vif.drop('summer',axis=1)

    #rebuild model
    lm2 = build_model(X_train_vif1)

    # check the VIF for our new model
    _ = VIF(X_train_vif1)
except:
    print("summer is not available in the train dataset")

**Observation**

The model give a decent R-square even after dorpping so lets pick another feature to drop

In [ ]:
try:
    # dropping winter from the dataset for testing the model output
    X_train_vif2 = X_train_vif1.drop('winter',axis=1)

    # rebuild model
    lm3 = build_model(X_train_vif2)

    # check the VIF for our new model
    _ = VIF(X_train_vif2)
except:
    print("winter is not available in the train dataset")    

In [ ]:
X_train_vif2.shape

**Observation**

The model looks good with 8 features , the **r-Square** value is above 0.8 which i assume to be a good model and on comparing the value of **F-Staticstics** it has a good value compared to other models.

Wiith this final model lets proceed to Prediction and residual analysis.

# Final Model

Now we need to know if the error terms are normally distributed lets plot a histogram and check

## Using Sklearn library for prediction

For Prediction and error we can use **Sklearn** model as it has many validation terms.

We already know the best features for our model which was derieved from our best model **lm3** so we can take and use features  from that model with X_train_vif2 as final data set

In [ ]:
# creating linar regressor
lm_final = LinearRegression()

# Initializing our model with the best features for prediction
# which was obtained from lm3 model
lm_final.fit(X_train_vif2,y_train)

# Predcting values in our train data set
y_train_pred = lm_final.predict(X_train_vif2)

### Prediction and Residual Analysis for the final model on Trainset

In [ ]:
# Plot the histogram for the error terms
sns.distplot((y_train - y_train_pred), bins = 20)
# set title for plot
plt.title('Error Terms', fontsize = 20)                  
# set x-axis label
plt.xlabel('Errors', fontsize = 18)                     
plt.show()

In [ ]:
print('Intercept:', lm_final.intercept_)
print('Coefficients:', lm_final.coef_)
print('Mean squared error (MSE): {:.2f}'.format(mean_squared_error(y_train, y_train_pred)))
print('Coefficient of Determination (R2): {:.2f}'.format(r2_score(y_train, y_train_pred)))

# Validating model on testset

In [ ]:
# Scaling test datset
df_test[numerical_features] = scaler.transform(df_test[numerical_features])

In [ ]:
# Extracting target column
y_test = df_test.pop('cnt')

# taking predictor feactures for test set
X_test = df_test

In [ ]:
# filetiring our test data set with the best columns for prediction
X_test_vif = X_test[X_train_vif2.columns]

In [ ]:
# making prediction on the final model
y_test_pred = lm_final.predict(X_test_vif)

## Model evaluation on testset

In [ ]:
y_test

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_test_pred)
# Plot title 
plt.title('y_test vs y_test_pred', fontsize=20,pad=10)
# plot X-label and y-label
plt.xlabel('y_test', fontsize=18)                          
plt.ylabel('y_test_pred', fontsize=16)    

In [ ]:
print('Trainset Mean squared error (MSE): {:.2f}'.format(mean_squared_error(y_train, y_train_pred)))
print('Trainset Coefficient of Determination (R2): {:.2f}'.format(r2_score(y_train, y_train_pred)))
print('\nTestset Mean squared error (MSE): {:.2f}'.format(mean_squared_error(y_test, y_test_pred)))
print('Testset Coefficient of Determination (R2): {:.2f}'.format(r2_score(y_test, y_test_pred)))

In [ ]:
# lm3 is the best model
print(lm3.summary())

<h1> Model Conslusion </h1><br>
<h2>
Train r2 score : 0.82<br/>    
    
Test r2 score : 0.80
</h2>
<h3>
We can conclude with the best fit line as:
</h3>
<h4>    
cnt= const * 0.218 + atemp * 0.3880 + yr * 0.2345 + Sep * 0.0695 + workingday * 0.0565 + Sat * 0.0628 + spring * -0.1624 + rain * -0.2943 + cloudy * -0.0780
</h4>

The demand of bikes is mainly influenced by actual temperature,year, september, workingday, saturday.<br><br>
The demand of bikes mostly decreases with attributes spring, light rain/light snow and cloudy